In [1]:
import pandas as pd
import numpy as np
import math
import scipy.stats as st

In [2]:
# Load dataset
data = pd.read_csv('titanic_data.csv')
print(data.describe())
print('- - - - - - - - - - - - - - - - - - - -')

         Survived      Pclass         Sex         Age  \
count  887.000000  887.000000  887.000000  887.000000   
mean     0.385569    2.305524    0.354002   29.471443   
std      0.487004    0.836662    0.478480   14.121908   
min      0.000000    1.000000    0.000000    0.420000   
25%      0.000000    2.000000    0.000000   20.250000   
50%      0.000000    3.000000    0.000000   28.000000   
75%      1.000000    3.000000    1.000000   38.000000   
max      1.000000    3.000000    1.000000   80.000000   

       Siblings/Spouses Aboard  Parents/Children Aboard       Fare  
count               887.000000               887.000000  887.00000  
mean                  0.525366                 0.383315   32.30542  
std                   1.104669                 0.807466   49.78204  
min                   0.000000                 0.000000    0.00000  
25%                   0.000000                 0.000000    7.92500  
50%                   0.000000                 0.000000   14.45420  
75%

In [3]:
# Variable transform
CATEGORICAL_FEATURES = ['Survived', 'Pclass', 'Sex', 'Siblings/Spouses Aboard', 'Parents/Children Aboard']
NUMERICAL_FEATURES = ['Age', 'Fare']
ALL_FEATURES = CATEGORICAL_FEATURES + NUMERICAL_FEATURES
for feature_name in CATEGORICAL_FEATURES:
    data = data.join(pd.get_dummies(data[feature_name], prefix=feature_name))
    data.drop([feature_name], axis = 1, inplace=True)
for feature_name in NUMERICAL_FEATURES:
    data = data.join(pd.get_dummies(pd.qcut(data[feature_name], 5), prefix=feature_name))
    data.drop([feature_name], axis = 1, inplace=True)
features = list(data.columns)
print('First, I divided these six features into two categories.')
print('CATEGORICAL_FEATURES and NUMERICAL_FEATURES')
print('For CATEGORICAL_FEATURES, I directly convert it to one-hot encoding.')
print('For NUMERICAL_FEATURES, binning the data and then convert it to one-hot encoding.')
print(data.describe())
print('- - - - - - - - - - - - - - - - - - - -')

First, I divided these six features into two categories.
CATEGORICAL_FEATURES and NUMERICAL_FEATURES
For CATEGORICAL_FEATURES, I directly convert it to one-hot encoding.
For NUMERICAL_FEATURES, binning the data and then convert it to one-hot encoding.
       Survived_0  Survived_1    Pclass_1    Pclass_2    Pclass_3       Sex_0  \
count  887.000000  887.000000  887.000000  887.000000  887.000000  887.000000   
mean     0.614431    0.385569    0.243517    0.207441    0.549042    0.645998   
std      0.487004    0.487004    0.429447    0.405703    0.497870    0.478480   
min      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
25%      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
50%      1.000000    0.000000    0.000000    0.000000    1.000000    1.000000   
75%      1.000000    1.000000    0.000000    0.000000    1.000000    1.000000   
max      1.000000    1.000000    1.000000    1.000000    1.000000    1.000000   

            Sex_1 

In [4]:
# Entropy
def Entropy(FEATURE):
    ans = 0
    for col in features:
        if col[:len(FEATURE)] == FEATURE:
            pr = data[col].sum()/data[col].count()
            ans = ans - pr * math.log(pr)
    return ans

# Conditional Entropy
def H(FEATURE1, FEATURE2):
    ans = 0
    for col_y in features:
        if col_y[:len(FEATURE2)] == FEATURE2:
            Pr_val = data[col_y].sum() / data[col_y].count()
            sub_ans = 0
            for col_x in features:
                if col_x[:len(FEATURE1)] == FEATURE1:
                    pr = np.sum(data[col_x]+data[col_y] == 2) / data[col_y].sum()
                    if pr == 0:
                        continue
                    sub_ans = sub_ans - pr * math.log(pr)
            ans = ans + Pr_val * sub_ans
    return ans

# Mutual Information
def I(FEATURE1, FEATURE2):
    return Entropy(FEATURE1) - H(FEATURE1, FEATURE2)

In [5]:
# Find Max_Mutual_Information_feature
def Find_Max_Mutual_Information_feature(DATA, ALL_FEATURES):
    y_label = ALL_FEATURES[0]
    Max_Mutual_Information_feature_name = ALL_FEATURES[1]
    Max_Mutual_Information = I(Max_Mutual_Information_feature_name, y_label)
    for feature_name in ALL_FEATURES[1:]:
        if I(feature_name, y_label) > Max_Mutual_Information:
            Max_Mutual_Information_feature_name = feature_name
            Max_Mutual_Information = I(feature_name, y_label)
    return Max_Mutual_Information_feature_name

In [8]:
# Split Data
Selected_Feature = Find_Max_Mutual_Information_feature(data, ALL_FEATURES)
New_ALL_FEATURES = [Feature for Feature in ALL_FEATURES if Feature != Selected_Feature]
vals = [val for val in features if val[:len(Selected_Feature)] == Selected_Feature]
for val in vals:
    New_data = data[data[val] == 1].drop(vals, axis=1)
    New_Selected_Feature = Find_Max_Mutual_Information_feature(New_data, New_ALL_FEATURES)
    print(New_Selected_Feature)

Pclass
Pclass


In [24]:
class Node:
    def __init__(self, feature):
        self.feature = feature
        self.children = []
        self.children_num = 0
    def Insert(self, val, feature):
        Child = Node(feature)
        self.children.append(Child)
        self.children_num = self.children_num + 1 

In [28]:
Root = Node('sex')
Root.Insert(0, 'Pclass')
Root.Insert(3, 'Fare')
Root.Insert(2, 'Age')

In [32]:
Root.children_num

3